In [1]:
import pandas as pd
import pymongo
from numba import vectorize,float64
import numpy as np
from scipy.stats import chi2_contingency,percentileofscore
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [3]:
genes = pd.Series([doc["_id"] for doc in db.genes.descs.ancestors.aggregate([{"$group":{"_id":"$gene_id"}}])])
# genes = pd.Series([doc["_id"] for doc in db.genes.descs.original.aggregate([{"$group":{"_id":"$gene_id"}}])])

In [4]:
uids = pd.Series([doc["_id"] for doc in db.genes.descs.ancestors.aggregate([{"$group":{"_id":"$uid"}}])])
# uids = pd.Series([doc["_id"] for doc in db.genes.descs.original.aggregate([{"$group":{"_id":"$uid"}}])])

In [5]:
pmids = pd.Series([doc["_id"] for doc in db.genes.descs.ancestors.aggregate([{"$group":{"_id":"$pmid"}}])])
# pmids = pd.Series([doc["_id"] for doc in db.genes.descs.original.aggregate([{"$group":{"_id":"$pmid"}}])])

In [6]:
df = pd.DataFrame([{"gene_id":doc["_id"]["gene_id"],
                    "uid":doc["_id"]["uid"],
                    "gd_freq":doc["count"]
                   } for doc in db.genes.descs.original.aggregate(
    [{"$group":{"_id":{"uid":"$uid","gene_id":"$gene_id"},"count":{"$sum":1}}}],allowDiskUse=True)])
# df = pd.DataFrame([{"gene_id":doc["_id"]["gene_id"],
#                     "uid":doc["_id"]["uid"],
#                     "gd_freq":doc["count"]
#                    } for doc in db.genes.descs.original.aggregate(
#     [{"$group":{"_id":{"uid":"$uid","gene_id":"$gene_id"},"count":{"$sum":1}}}],allowDiskUse=True)])

In [7]:
g_freq = pd.DataFrame([{"gene_id":doc["_id"],"g_freq":doc["g_freq"]} 
              for doc in db.genes.descs.original.aggregate([{"$group":{"_id":"$gene_id","g_freq":{"$sum":1}}}])])
# g_freq = pd.DataFrame([{"gene_id":doc["_id"],"g_freq":doc["g_freq"]} 
#               for doc in db.genes.descs.original.aggregate([{"$group":{"_id":"$gene_id","g_freq":{"$sum":1}}}])])

In [8]:
d_freq = pd.DataFrame([{"uid":doc["_id"],"d_freq":doc["d_freq"]} 
              for doc in db.genes.descs.ancestors.aggregate([{"$group":{"_id":"$uid","d_freq":{"$sum":1}}}])])
# d_freq = pd.DataFrame([{"uid":doc["_id"],"d_freq":doc["d_freq"]} 
#               for doc in db.genes.descs.original.aggregate([{"$group":{"_id":"$uid","d_freq":{"$sum":1}}}])])

In [9]:
gene_uid = df.merge(g_freq).merge(d_freq)

In [10]:
gene_uid.to_pickle("../data/SSI/gene_uid.p")
# gene_uid.to_pickle("../data/SSI/gene_uid_original.p")